## Q1-1)

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import random

In [2]:
# 파라미터 설정 (learning rate, training epochs, batch_size)
learning_rate = 0.1
training_epochs = 15
batch_size = 100

In [3]:
#train set과 test set 나누어서 불러오기
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [4]:
#dataset loader에 train과 test할당하기(batch size, shuffle, drop_last 잘 설정할 것!)
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)

In [5]:
# Layer 쌓기 (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)
# 각 Layer의 Hidden node 수 : 1st Layer (784,100), 2nd Layer(100,100),3rd Layer(100,10)
linear1=torch.nn.Linear(784,100)
linear2=torch.nn.Linear(100,100)
linear3=torch.nn.Linear(100,10)

dropout = torch.nn.Dropout(p=0.3)

relu=torch.nn.ReLU()

bn1=torch.nn.BatchNorm1d(100)
bn2=torch.nn.BatchNorm1d(100)


In [6]:
#xavier initialization을 이용하여 각 layer의 weight 초기화

torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)


Parameter containing:
tensor([[-0.1734,  0.2040,  0.0221,  0.0829,  0.2211, -0.0367,  0.1658,  0.1042,
          0.1742, -0.0707, -0.0363,  0.1077,  0.1091,  0.1657,  0.0090,  0.0979,
         -0.0237, -0.0873, -0.1394,  0.1187,  0.1299, -0.1470,  0.0780,  0.0291,
          0.0050,  0.0103, -0.1480,  0.2285,  0.0913,  0.0559,  0.1850, -0.1609,
          0.2012,  0.0269,  0.1427,  0.0512,  0.0652, -0.2246,  0.2166,  0.0095,
         -0.0218, -0.0104, -0.1898,  0.1537, -0.0284, -0.2309,  0.2303, -0.0937,
         -0.1007,  0.1054,  0.1868,  0.1807,  0.1922,  0.1272, -0.0015,  0.1771,
          0.1740, -0.2284,  0.1472, -0.0109, -0.1430,  0.1879,  0.0008, -0.1404,
          0.1173, -0.2211, -0.2128,  0.2255, -0.1946,  0.0614, -0.2046, -0.0526,
         -0.0140, -0.1702,  0.1584, -0.0005,  0.1556,  0.1162,  0.0061, -0.2322,
         -0.2224, -0.0422, -0.1489,  0.0529, -0.0744, -0.0817,  0.1051, -0.1647,
         -0.0159,  0.1319, -0.2163,  0.1371,  0.0791, -0.0847,  0.0275,  0.1335,
      

In [7]:
# torch.nn.Sequential을 이용하여 model 정의하기(쌓는 순서: linear-Batch Normalization Layer - ReLU- DropOut)
model = torch.nn.Sequential(linear1, bn1, relu, dropout,linear2, bn2, relu, dropout, linear3)

In [8]:
# Loss Function 정의하기 (CrossEntropy를 사용할 것!)
criterion=torch.nn.CrossEntropyLoss()
#optimizer 정의하기 (Adam optimizer를 사용할 것!)
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)

In [10]:
#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 
for epoch in range(training_epochs):
    model.train()
    avg_cost=0
    
#train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
    for X, Y in train_loader:
        X = X.view(-1, 28 * 28)
        Y = Y

        optimizer.zero_grad()
        prediction = model(X)
        loss = criterion(prediction, Y)
        loss.backward()
        optimizer.step()
        avg_cost += loss / train_total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

        

Epoch: 0001 cost = 0.498689592
Epoch: 0002 cost = 0.371965975
Epoch: 0003 cost = 0.338367999
Epoch: 0004 cost = 0.302482396
Epoch: 0005 cost = 0.297170192
Epoch: 0006 cost = 0.284864366
Epoch: 0007 cost = 0.282527566
Epoch: 0008 cost = 0.278607279
Epoch: 0009 cost = 0.259173632
Epoch: 0010 cost = 0.259774745
Epoch: 0011 cost = 0.254910409
Epoch: 0012 cost = 0.255829126
Epoch: 0013 cost = 0.240515888
Epoch: 0014 cost = 0.239837259
Epoch: 0015 cost = 0.239134982
Learning finished


In [14]:
#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    model.eval()
    
    X_test = mnist_test.test_data.view(-1, 28 * 28).float()
    Y_test = mnist_test.test_labels

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1)==Y_test
    acc = correct_prediction.float().mean()
    print("Accuracy: ", acc.item())
    
     ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())


Accuracy:  0.9315000176429749
Label:  7
Prediction:  7


## Q1-2)

In [23]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import random

learning_rate = 0.1
training_epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)

In [28]:

w_linear1=torch.nn.Linear(784,350)
w_linear2=torch.nn.Linear(350,150)
w_linear3=torch.nn.Linear(150,10)
dropout = torch.nn.Dropout(p=0.3)
relu=torch.nn.ReLU()
bn1=torch.nn.BatchNorm1d(350)
bn2=torch.nn.BatchNorm1d(150)



In [29]:
torch.nn.init.xavier_uniform_(w_linear1.weight)
torch.nn.init.xavier_uniform_(w_linear2.weight)
torch.nn.init.xavier_uniform_(w_linear3.weight)

Parameter containing:
tensor([[-0.1081,  0.0570, -0.1428,  ..., -0.1672,  0.1690,  0.1234],
        [-0.0508, -0.1914, -0.1333,  ..., -0.0103,  0.1287, -0.1543],
        [-0.1072,  0.0993,  0.0144,  ..., -0.0063, -0.1570,  0.0284],
        ...,
        [ 0.1629,  0.1409,  0.1243,  ...,  0.1090,  0.0985, -0.0090],
        [ 0.0698,  0.0843,  0.1059,  ...,  0.1033,  0.1689,  0.1600],
        [ 0.0010, -0.0058, -0.1603,  ...,  0.1752,  0.0054, -0.0565]],
       requires_grad=True)

In [30]:
w_model = torch.nn.Sequential(w_linear1, bn1, relu, dropout,w_linear2, bn2, relu, dropout, w_linear3)

criterion=torch.nn.CrossEntropyLoss()

optimizer=torch.optim.Adam(w_model.parameters(), lr=learning_rate)

In [31]:
train_total_batch = len(train_loader)

for epoch in range(training_epochs):
    w_model.train()
    avg_cost=0
    
    for X, Y in train_loader:
        X = X.view(-1, 28 * 28)
        Y = Y

        optimizer.zero_grad()
        prediction = w_model(X)
        loss = criterion(prediction, Y)
        loss.backward()
        optimizer.step()
        avg_cost += loss / train_total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.434796959
Epoch: 0002 cost = 0.303037107
Epoch: 0003 cost = 0.271271706
Epoch: 0004 cost = 0.254768103
Epoch: 0005 cost = 0.236294970
Epoch: 0006 cost = 0.222446769
Epoch: 0007 cost = 0.215036392
Epoch: 0008 cost = 0.197678506
Epoch: 0009 cost = 0.214336842
Epoch: 0010 cost = 0.188481390
Epoch: 0011 cost = 0.193039224
Epoch: 0012 cost = 0.179785520
Epoch: 0013 cost = 0.171679839
Epoch: 0014 cost = 0.183403775
Epoch: 0015 cost = 0.172819927
Learning finished


In [42]:
with torch.no_grad():
    w_model.eval()
    
    X_test = mnist_test.test_data.view(-1, 28 * 28).float()
    Y_test = mnist_test.test_labels

    prediction = w_model(X_test)
    correct_prediction = torch.argmax(prediction,1)==Y_test
    acc = correct_prediction.float().mean()
    print("Accuracy: ", acc.item())
    
     ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = w_model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())


Accuracy:  0.8891000151634216
Label:  8
Prediction:  0
